In [1]:
%cd ../../

/mnt/sda/otani/Experiments/eval-detection


In [2]:
from jupyter_bbox_widget import BBoxWidget
import numpy as np
from src.extensions.metrics.ot_cost import get_ot_cost, get_cmap
import ot
import ipywidgets as widgets
import matplotlib.pyplot as plt
from src.extensions.metrics.ot_cost import get_bbox_overlaps
from matplotlib.patches import Rectangle
from matplotlib.pyplot import cm
%matplotlib inline

In [3]:
def display_alignment(cmap_fnc, use_dummy=False):
    gt_bboxes = [b for b in bbox_widget.bboxes if b["label"].startswith("GT")]
    fmt_gt_bboxes, gt_bbox_indices = format_bboxes(gt_bboxes, CLASS_LABELS, return_orders=True)
    bboxes = [b for b in bbox_widget.bboxes if not b["label"].startswith("GT")]
    fmt_bboxes, bbox_indices = format_bboxes(bboxes, CLASS_LABELS, return_orders=True)
    otc, log = get_ot_cost(fmt_gt_bboxes, fmt_bboxes, cmap_fnc, return_matrix=True)

    img = plt.imread("data/sample_img/white.jpeg")
    plt.imshow(img)
    ax = plt.gca()

    # draw gt bboxes
    color_map = plt.get_cmap("tab20")
    for i, bi in enumerate(gt_bbox_indices):
        b = gt_bboxes[bi]
        c = color_map(i*2)
        rect = Rectangle((b["x"], b["y"]), b["width"], b["height"], lw=2, ec=c, fc=c, alpha=0.7)
        ax.add_patch(rect)   

    # draw bboxes
    G = log["G"]
    eps = 0.05
    for i, weight in enumerate(G):
        for bi, w in zip(bbox_indices, weight):
            if w>0:
                n_w = w / (G[:, bi].sum() + eps)
                b = bboxes[bi]
                if use_dummy:
                    c = color_map(i*2+1) if i < len(G)-1 else "tab:gray"
                else:
                    c = color_map(i*2+1)
                rect = Rectangle((b["x"], b["y"]), b["width"], b["height"], lw=2, ec=c, fc="None", alpha=n_w)
                ax.add_patch(rect)   

    plt.show()
    return log

In [10]:
from IPython.display import display

CLASS_LABELS = ["GT", "Pred"]
n_class = len(CLASS_LABELS)
img_file = 'data/sample_img/fruits.jpeg'


bbox_widget = BBoxWidget(
    image="../../"+img_file,
    classes= CLASS_LABELS,
    colors=["orange", "green"],
    hide_buttons=True
    )

w_conf = widgets.FloatSlider(value=0.5, min=0, max=1., description='Confidence')
bbox_widget.attach(w_conf, name="confidence")

# def format_bboxes(bboxes):
#     formatted_bboxes = []
#     for i, bbox in enumerate(bboxes):
#         if bbox["label"] == "GT":
#             conf = 1 
#         else:
#             conf = bbox["confidence"]
#         formatted_bboxes.append([bbox["x"], bbox["y"], bbox["x"]+bbox["width"], bbox["y"]+bbox["height"], conf])
#     return np.asarray(formatted_bboxes, dtype=np.float32).reshape(-1, 5)

def format_bboxes(bboxes, classes, return_orders=False):
    orders = []
    formatted_bboxes = []
    for label in classes:
        formatted_bboxes.append([])
        for i, bbox in enumerate(bboxes):
            if label in bbox["label"]:
                if bbox["label"].startswith("GT"):
                    conf = 1 
                else:
                    conf = bbox["confidence"]
                formatted_bboxes[-1].append([bbox["x"], bbox["y"], bbox["x"]+bbox["width"], bbox["y"]+bbox["height"], conf])
                orders.append(i)
        formatted_bboxes[-1] = np.asarray(formatted_bboxes[-1], dtype=np.float32).reshape(-1, 5)
    if return_orders:
        return formatted_bboxes, orders
    return formatted_bboxes

def evaluate_bboxes():
    gt_bboxes = [b for b in bbox_widget.bboxes if b["label"]=="GT"]
    gt_bboxes = format_bboxes(gt_bboxes, ["GT", "Pred"])
    bboxes = [b for b in bbox_widget.bboxes if not b["label"]!="Pred"]
    bboxes = format_bboxes(bboxes,  ["GT", "Pred"])
    ious = get_bbox_overlaps(bboxes[:, :4], gt_bboxes[:, :4], mode="iou")
    return ious

w_out = widgets.Output()

# def update_label_conf():
#     idx = bbox_widget.selected_index
#     cur_label = bbox_widget.bboxes[idx]["label"]

#     if cur_label=="GT":
#         return
    
#     conf = bbox_widget.bboxes[idx]["confidence"]
#     new_label = f"Pred ({conf})"
#     bbox_widget._set_bbox_property(idx, "label", new_label)

def on_bbox_change(change):
    # update_label_conf()

    w_out.clear_output(wait=True)
    ious = evaluate_bboxes()
    with w_out:
        print(ious)
        
bbox_widget.observe(on_bbox_change, names=['bboxes'])

w_container = widgets.VBox([
    bbox_widget,
    w_conf,
    w_out,
])
display(w_container)

In [7]:
cmap_fnc = lambda x, y: get_cmap(x, y, use_dummy=True)
display_alignment(cmap_fnc, use_dummy=True)

TypeError: format_bboxes() got an unexpected keyword argument 'return_orders'

In [8]:
cmap_fnc = lambda x, y: get_cmap(x, y, use_dummy=False)
display_alignment(cmap_fnc)

TypeError: format_bboxes() got an unexpected keyword argument 'return_orders'